In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np

In [2]:
import sys
sys.path.append('../..')

In [96]:
from dstableFGM import dstableFGM, projectPSD, graddstableform, dstable_descent

In [106]:
A = np.random.normal(size=(50, 50), scale=2/np.sqrt(50))

In [107]:
np.linalg.eigvals(A)

array([-0.563027  +1.99923481j, -0.563027  -1.99923481j,
       -1.0853649 +1.59586755j, -1.0853649 -1.59586755j,
        2.02065694+0.j        ,  1.60264599+1.02897205j,
        1.60264599-1.02897205j,  1.78368554+0.24053984j,
        1.78368554-0.24053984j,  1.67458385+0.j        ,
       -0.13282776+1.745636j  , -0.13282776-1.745636j  ,
        1.0630318 +1.23108326j,  1.0630318 -1.23108326j,
       -1.49950233+0.53975902j, -1.49950233-0.53975902j,
       -1.15909632+1.03011307j, -1.15909632-1.03011307j,
       -0.82821156+1.19073674j, -0.82821156-1.19073674j,
        0.18162328+1.47460271j,  0.18162328-1.47460271j,
       -1.30485303+0.2474526j , -1.30485303-0.2474526j ,
       -1.32567403+0.j        , -1.00929688+0.66125941j,
       -1.00929688-0.66125941j,  0.4499974 +1.34105183j,
        0.4499974 -1.34105183j,  0.2031062 +1.2455258j ,
        0.2031062 -1.2455258j ,  0.63030391+1.08888788j,
        0.63030391-1.08888788j,  1.09770629+0.35789276j,
        1.09770629-0.35789276j,

In [108]:
A = dstable_descent(A, maxiter=200, init='default')

73.51538512069948
> /home/akumar/nse/neural_control/dstableFGM.py(93)dstable_descent()
     91             e[i] = e_[-1]
     92             pdb.set_trace()
---> 93             delta_e = np.mean(np.diff(e)[max(0, i-5):i])
     94             print(e[i])
     95             S = Sn

array([], dtype=float64)


BdbQuit: 

In [94]:
A = dstableFGM(A)

74.30695086203905
Display of iteration number and error:
> /home/akumar/nse/neural_control/dstableFGM.py(171)dstableFGM()
    169         pdb.set_trace()
    170         # Display
--> 171         if display:
    172             if i % ndisplay == 0:
    173                 print('%2.0f:%2.3f - ',i ,e[i + 1])

9.327820096769747
[array(2.10078916e+14), array(1.09468501e+14), array(5.48950304e+13), array(1.9309674e+13), array(223.25758967), array(30.3139607)]


BdbQuit: 

(array([[ 0.9516448 -3.82132227e-205j, -0.0074041 -1.19203027e-205j,
          0.02645596+3.24155923e-205j, ...,  0.05701331+4.89295050e-205j,
         -0.09166026+4.44431050e-205j,  0.07481819+3.60646099e-205j],
        [-0.0074041 -1.19203027e-205j,  1.03521214-8.76353826e-205j,
         -0.06131114-1.66066035e-204j, ..., -0.06267445-1.06455763e-204j,
         -0.02376924+1.74438901e-205j,  0.05510339-2.29758374e-205j],
        [ 0.02645596+3.24155923e-205j, -0.06131114-1.66066035e-204j,
          1.04492004+3.42629166e-206j, ..., -0.03292833-5.98794572e-205j,
          0.06278156+1.35216262e-204j, -0.03512779-5.18630483e-205j],
        ...,
        [ 0.05701331+4.89295050e-205j, -0.06267445-1.06455763e-204j,
         -0.03292833-5.98794572e-205j, ...,  1.05577842-4.11563271e-206j,
          0.01141149+5.72136725e-205j,  0.03007406-3.36837073e-205j],
        [-0.09166026+4.44431050e-205j, -0.02376924+1.74438901e-205j,
          0.06278156+1.35216262e-204j, ...,  0.01141149+5.72136725

In [44]:
import torch
import scipy

In [43]:
# Step (1): Explicitly check the gradients
# Step (2): Check against the examples provided in the paper
# Step (3): Start many random initilizations

In [81]:
A = np.random.normal(size=(50, 50), scale=2/np.sqrt(50))
U, H = scipy.linalg.polar(A)
B = projectPSD(H, 1e-6, 1 - 1e-6)
At = torch.tensor(A, requires_grad=True)
S = np.diag(np.random.uniform(0.1, 0.9, size=(50,)))
St = torch.tensor(S, requires_grad=True)
Ut = torch.tensor(U, requires_grad=True)
Bt = torch.tensor(B, requires_grad=True)

In [82]:
loss = torch.pow(torch.norm(At - torch.chain_matmul(torch.inverse(St), Ut, Bt, St)), 2)

In [83]:
loss.backward()

In [85]:
St.grad

tensor([[ 3.0890e+00,  1.1302e+00, -5.1995e+00,  ..., -4.1789e-01,
         -4.1198e-01,  2.5708e+00],
        [-1.8304e+00,  1.2016e+01,  2.5484e-01,  ...,  6.0706e-03,
          2.5344e+00,  2.2524e+00],
        [-5.3120e+00, -2.4581e+00, -3.8214e+01,  ..., -9.8562e-01,
          2.8818e+00,  3.7360e+00],
        ...,
        [ 1.2736e+00,  1.7175e+00,  1.5691e+00,  ...,  2.9873e+00,
          4.6111e-01,  6.2369e-01],
        [-1.7762e+00,  4.5122e+00, -5.5682e-01,  ...,  1.4198e-01,
          6.7325e-01, -1.4837e-01],
        [ 8.6948e-01,  4.3131e+00,  9.8137e-01,  ...,  2.5350e+00,
          1.4256e+00, -4.7278e+00]], dtype=torch.float64)

In [86]:
e, gS, gU, gB = graddstableform(A, np.eye(50), U, B, return_grad=True)

In [88]:
np.linalg.norm(gS - St.grad.detach().numpy())

524.993263304741

In [80]:
np.linalg.norm(gU - Ut.grad.detach().numpy())

34.09716910329542